# Zindi DSN Challenge: Financial Inclusion

In [ ]:
!pip install catboost -qq

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
variable_description =  pd.read_csv('Assignment2_data/VariableDescription.csv', index_col='Variable')
variable_description.head()

SyntaxError: ignored

In [ ]:
def get_info(col):
    print(variable_description.loc[col]['Description'])
get_info('Residential')

is the building a residential building or not


# Load Data

In [ ]:
Train = pd.read_csv('Assignment2_data/train_data.csv')
Test = pd.read_csv('Assignment2_data/test_data.csv')

submit = pd.read_csv('Assignment2_data/sample_submission.csv')

train = Train.copy()
test = Test.copy()

In [ ]:
train.head()

In [ ]:
train.shape, test.shape

In [ ]:
#alll_columns = train.columns[:-1]
#train.info()

In [ ]:
all_columns

Index(['Customer Id', 'YearOfObservation', 'Insured_Period', 'Residential',
       'Building_Painted', 'Building_Fenced', 'Garden', 'Settlement',
       'Building Dimension', 'Building_Type', 'Date_of_Occupancy',
       'NumberOfWindows', 'Geo_Code'],
      dtype='object')

In [ ]:
train.describe()

In [ ]:
train.corr()['Claim']
# These numerical features are quite good since there is little correlation btw them and the label

In [ ]:
train.isnull().sum()

# EDA - Visualization

In [ ]:
train['Claim'].value_counts()

In [ ]:
sns.countplot(train['Claim'])

In [ ]:
pd.crosstab(train['YearOfObservation'], train['Claim']).plot.bar()

In [ ]:
sns.heatmap(train.corr(), cmap='viridis', annot=True)

# Feature Engineering

Generation

In [ ]:
get_info('YearOfObservation')

In [ ]:
train['No_of_years'] = 2020 - train['YearOfObservation']
test['No_of_years'] = 2020 - test['YearOfObservation']

In [ ]:
get_info('Insured_Period')

In [ ]:
train['Insured_period_bin'] = pd.cut(train['Insured_Period'], 3, [0, 0.5, 1])
test['Insured_period_bin'] = pd.cut(test['Insured_Period'], 3, [0, 0.5, 1])

In [ ]:
train['Current_InsuredPeriod_by_Year'] = train['Insured_Period'] * train['No_of_years']
test['Current_InsuredPeriod_by_Year'] = test['Insured_Period'] * test['No_of_years']

In [ ]:
get_info('Residential')

In [ ]:
get_info('Building Dimension')

In [ ]:
train['Building_dim_in_unit'] = np.sqrt(train['Building Dimension'])
test['Building_dim_in_unit'] = np.sqrt(test['Building Dimension'])

In [ ]:
get_info('Date_of_Occupancy')

In [ ]:
train['Occupied_till_now'] = 2020 - train['Date_of_Occupancy']
test['Occupied_till_now'] = 2020 - test['Date_of_Occupancy']

Preprocessing

In [ ]:
# Filling missing values
missing = train.isnull()
missing.sum()

In [ ]:
train['Garden'] = train['Garden'].fillna(train['Garden'].mode()[0])
test['Garden'] = test['Garden'].fillna(test['Garden'].mode()[0])

In [ ]:
train.corr()['Building Dimension'].sort_values()

In [ ]:
train['Building Dimension'] = train['Building Dimension'].fillna(train['Building Dimension'].mean())
test['Building Dimension'] = test['Building Dimension'].fillna(test['Building Dimension'].mean())

train['Building_dim_in_unit'] = train['Building_dim_in_unit'].fillna(train['Building_dim_in_unit'].mean())
test['Building_dim_in_unit'] = test['Building_dim_in_unit'].fillna(test['Building_dim_in_unit'].mean())

In [ ]:
train.isnull().sum()

In [ ]:
train['Date_of_Occupancy'] = train['Date_of_Occupancy'].fillna(int(train['Date_of_Occupancy'].mean()))
test['Date_of_Occupancy'] = test['Date_of_Occupancy'].fillna(int(test['Date_of_Occupancy'].mean()))

train['Occupied_till_now'] = train['Occupied_till_now'].fillna(int(train['Occupied_till_now'].mean()))
test['Occupied_till_now'] = test['Occupied_till_now'].fillna(int(test['Occupied_till_now'].mean()))

In [ ]:
train['YearOfObservation'].sort_values().unique()

In [ ]:
train['Building_Painted'].sort_values().unique()

In [ ]:
train['Building_Fenced'].sort_values().unique()

In [ ]:
train['Building_Type'].sort_values().unique()

In [ ]:
train['Date_of_Occupancy'].sort_values().unique()

In [ ]:
print(train['NumberOfWindows'].sort_values().unique())
get_info('NumberOfWindows')

In [ ]:
train['Geo_Code'].sort_values().nunique()

In [ ]:
train.info()

In [ ]:
sns.countplot(train['Building_Painted'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
for i in range(4,8):
    train.iloc[:, i] = encoder.fit_transform(train.iloc[:, i])
for i in range(4,8):
    test.iloc[:, i] = encoder.fit_transform(test.iloc[:, i])

In [ ]:
train.iloc[:, 4:8].head()

In [ ]:
train['Insured_period_bin'] = encoder.fit_transform(train['Insured_period_bin'])
test['Insured_period_bin'] = encoder.fit_transform(test['Insured_period_bin'])

In [ ]:
pd.crosstab(train['NumberOfWindows'], train['Claim']).plot.bar()

In [ ]:
train['NumberOfWindows'] = encoder.fit_transform(train['NumberOfWindows'])
test['NumberOfWindows'] = encoder.fit_transform(test['NumberOfWindows'])

Feature Selection

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.shape, test.shape

In [ ]:
train = train.drop(['Customer Id', 'Geo_Code'], 1)
test = test.drop(['Customer Id', 'Geo_Code'], 1)

In [ ]:
X = train.drop('Claim', 1)
y = train['Claim']

# Modeling

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

In [ ]:
kfold = KFold(n_splits=5, random_state=101)
lr = LogisticRegression()

valid = []
test_pred = np.zeros(len(test))

In [ ]:
for fold, (tr_in, te_in) in enumerate(kfold.split(X)):
    X_train, y_train = X.iloc[tr_in], y.iloc[tr_in]
    X_test, y_test = X.iloc[te_in], y.iloc[te_in]
    
    lr.fit(X_train, y_train)
    
    valid.append(roc_auc_score(lr.predict(X_test), y_test))
    test_pred += lr.predict(test)
    print(np.mean(valid))
    
print(np.mean(valid))

In [ ]:
test_pred /= 5

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
rfc = RandomForestClassifier(n_estimators=100, random_state=101, max_depth=4, bootstrap=True)

In [ ]:
test_pred2 = np.zeros(len(test))
for fold, (tr_in, te_in) in enumerate(kfold.split(X)):
    X_train, y_train = X.iloc[tr_in], y.iloc[tr_in]
    X_test, y_test = X.iloc[te_in], y.iloc[te_in]
    
    rfc.fit(X_train, y_train)
    
    valid.append(roc_auc_score(rfc.predict(X_test), y_test))
    test_pred2 += rfc.predict(test)
    
    print(np.mean(valid), '\n')
    print(classification_report(rfc.predict(X_test), y_test), '\n')
    print(confusion_matrix(rfc.predict(X_test), y_test), '\n')
    
print(np.mean(valid))

In [ ]:
feat_imp = pd.Series(X.columns,rfc.feature_importances_)
feat_imp

0.005866                YearOfObservation
0.038111                   Insured_Period
0.003890                      Residential
0.000840                 Building_Painted
0.001643                  Building_Fenced
0.000615                           Garden
0.001847                       Settlement
0.371319               Building Dimension
0.045822                    Building_Type
0.026124                Date_of_Occupancy
0.061786                  NumberOfWindows
0.005344                      No_of_years
0.015121               Insured_period_bin
0.037268    Current_InsuredPeriod_by_Year
0.360176             Building_dim_in_unit
0.024226                Occupied_till_now
dtype: object

In [ ]:
print(classification_report(rfc.predict(X_test), y_test))

             precision    recall  f1-score   support

          0       0.97      0.78      0.86      1355
          1       0.12      0.53      0.20        77

avg / total       0.92      0.77      0.83      1432



In [ ]:
gb = GradientBoostingClassifier(n_estimators=100, random_state=101, max_depth=4, learning_rate=0.01)

In [ ]:
test_pred2 = np.zeros(len(test))
for fold, (tr_in, te_in) in enumerate(kfold.split(X)):
    X_train, y_train = X.iloc[tr_in], y.iloc[tr_in]
    X_test, y_test = X.iloc[te_in], y.iloc[te_in]
    
    gb.fit(X_train, y_train, valid)
    
    valid.append(roc_auc_score(gb.predict(X_test), y_test))
    test_pred2 += gb.predict(test)
    
    print(np.mean(valid), '\n')
    print(classification_report(gb.predict(X_test), y_test), '\n')
    print(confusion_matrix(gb.predict(X_test), y_test), '\n')
    
print(np.mean(valid))

SystemError: ignored

Exception ignored in: '_catboost._MetricEval'
Traceback (most recent call last):
  File "_catboost.pyx", line 1230, in _catboost._DoubleArrayWrapper.create
SystemError: <class '_catboost._DoubleArrayWrapper'> returned a result with an error set


CatBoostError: ignored

In [ ]:
feat_imp = pd.Series(X.columns,rfc.feature_importances_)
feat_imp

0.005866                YearOfObservation
0.038111                   Insured_Period
0.003890                      Residential
0.000840                 Building_Painted
0.001643                  Building_Fenced
0.000615                           Garden
0.001847                       Settlement
0.371319               Building Dimension
0.045822                    Building_Type
0.026124                Date_of_Occupancy
0.061786                  NumberOfWindows
0.005344                      No_of_years
0.015121               Insured_period_bin
0.037268    Current_InsuredPeriod_by_Year
0.360176             Building_dim_in_unit
0.024226                Occupied_till_now
dtype: object